# Lab 3.3.6 Solutions: TensorRT-LLM Optimization

Complete solutions to all exercises from the TensorRT-LLM optimization lab.

## Setup

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path("../scripts").resolve()))

import numpy as np
import time
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import json

---

## Exercise 1: Calculate Optimal Quantization Settings

**Task**: Analyze memory and performance trade-offs for different quantization strategies.

In [ ]:
@dataclass
class QuantizationConfig:
    """Configuration for model quantization."""
    name: str
    weight_bits: int
    activation_bits: int
    kv_cache_bits: int
    compute_dtype: str
    supported_on_spark: bool  # DGX Spark Blackwell GPU

# Available quantization options
QUANT_CONFIGS = {
    "fp16": QuantizationConfig(
        name="FP16", weight_bits=16, activation_bits=16, 
        kv_cache_bits=16, compute_dtype="float16", supported_on_spark=True
    ),
    "bf16": QuantizationConfig(
        name="BF16", weight_bits=16, activation_bits=16,
        kv_cache_bits=16, compute_dtype="bfloat16", supported_on_spark=True
    ),
    "fp8": QuantizationConfig(
        name="FP8", weight_bits=8, activation_bits=8,
        kv_cache_bits=8, compute_dtype="float8_e4m3fn", supported_on_spark=True
    ),
    "int8": QuantizationConfig(
        name="INT8", weight_bits=8, activation_bits=8,
        kv_cache_bits=8, compute_dtype="int8", supported_on_spark=True
    ),
    "int4": QuantizationConfig(
        name="INT4 (AWQ)", weight_bits=4, activation_bits=16,
        kv_cache_bits=8, compute_dtype="int4_awq", supported_on_spark=True
    ),
    "nvfp4": QuantizationConfig(
        name="NVFP4 (Blackwell)", weight_bits=4, activation_bits=8,
        kv_cache_bits=4, compute_dtype="nvfp4", supported_on_spark=True
    ),
}


@dataclass
class ModelSpec:
    """Model specification for memory calculations."""
    name: str
    params_billions: float
    hidden_size: int
    num_layers: int
    num_kv_heads: int
    head_dim: int
    max_seq_len: int

MODELS = {
    "llama-8b": ModelSpec("Llama 3.1 8B", 8, 4096, 32, 8, 128, 131072),
    "llama-70b": ModelSpec("Llama 3.1 70B", 70, 8192, 80, 8, 128, 131072),
    "llama-405b": ModelSpec("Llama 3.1 405B", 405, 16384, 126, 8, 128, 131072),
}


def calculate_memory_requirements(model: ModelSpec, 
                                   quant: QuantizationConfig,
                                   batch_size: int = 1,
                                   seq_len: int = 4096) -> dict:
    """
    Calculate memory requirements for a model with specific quantization.
    """
    # Weight memory
    weight_bytes = model.params_billions * 1e9 * (quant.weight_bits / 8)
    weight_gb = weight_bytes / (1024**3)
    
    # KV cache memory per token
    kv_bytes_per_token = (
        2 *  # K and V
        model.num_layers *
        model.num_kv_heads *
        model.head_dim *
        (quant.kv_cache_bits / 8)
    )
    
    kv_cache_gb = (batch_size * seq_len * kv_bytes_per_token) / (1024**3)
    
    # Activation memory (approximate)
    activation_bytes = (
        batch_size * 
        seq_len * 
        model.hidden_size * 
        (quant.activation_bits / 8) *
        4  # Approximate multiplier for intermediate activations
    )
    activation_gb = activation_bytes / (1024**3)
    
    total_gb = weight_gb + kv_cache_gb + activation_gb
    
    # Check if fits in DGX Spark (128GB unified memory)
    dgx_spark_memory = 128
    fits_spark = total_gb < dgx_spark_memory * 0.9  # 90% utilization
    
    return {
        "weight_gb": weight_gb,
        "kv_cache_gb": kv_cache_gb,
        "activation_gb": activation_gb,
        "total_gb": total_gb,
        "fits_dgx_spark": fits_spark,
        "headroom_gb": dgx_spark_memory - total_gb
    }


def estimate_performance(quant: QuantizationConfig) -> dict:
    """
    Estimate relative performance for a quantization config.
    """
    # Baseline: FP16
    base_tflops = 1.0
    
    # Performance scaling based on quantization
    if quant.weight_bits == 16:
        throughput_multiplier = 1.0
    elif quant.weight_bits == 8:
        throughput_multiplier = 1.8  # ~1.8x faster
    elif quant.weight_bits == 4:
        throughput_multiplier = 2.5  # ~2.5x faster
    else:
        throughput_multiplier = 1.0
    
    # Quality impact (rough estimates)
    if quant.weight_bits >= 16:
        quality_impact = "None"
        perplexity_increase = 0.0
    elif quant.weight_bits == 8:
        quality_impact = "Minimal (<1%)"
        perplexity_increase = 0.02
    elif quant.weight_bits == 4:
        quality_impact = "Small (1-3%)"
        perplexity_increase = 0.05
    else:
        quality_impact = "Unknown"
        perplexity_increase = 0.1
    
    return {
        "throughput_multiplier": throughput_multiplier,
        "quality_impact": quality_impact,
        "perplexity_increase": perplexity_increase
    }


# Analyze quantization options
print("📊 TensorRT-LLM Quantization Analysis")
print("=" * 85)

model = MODELS["llama-70b"]
print(f"\nModel: {model.name} ({model.params_billions}B parameters)")
print(f"Target: DGX Spark (128GB unified memory)")
print(f"Batch size: 8, Sequence length: 4096\n")

print(f"{'Quant':<15} {'Weights':>10} {'KV Cache':>10} {'Total':>10} {'Fits?':>8} {'Speedup':>10} {'Quality':>15}")
print("-" * 85)

for name, quant in QUANT_CONFIGS.items():
    mem = calculate_memory_requirements(model, quant, batch_size=8, seq_len=4096)
    perf = estimate_performance(quant)
    
    fits = "✓" if mem["fits_dgx_spark"] else "✗"
    print(f"{quant.name:<15} {mem['weight_gb']:>8.1f}GB {mem['kv_cache_gb']:>8.1f}GB "
          f"{mem['total_gb']:>8.1f}GB {fits:>8} {perf['throughput_multiplier']:>9.1f}x "
          f"{perf['quality_impact']:>15}")

print("\n💡 Recommendations for DGX Spark:")
print("   • Llama 8B: Use BF16 (plenty of headroom)")
print("   • Llama 70B: Use FP8 or INT8 for best quality/speed balance")
print("   • Llama 405B: Requires NVFP4 or distributed inference")

---

## Exercise 2: Build Optimized TensorRT Engine

**Task**: Create a script to build and optimize a TensorRT-LLM engine.

In [ ]:
# TensorRT-LLM Build Script Generator

def generate_build_script(model_name: str,
                          model_path: str,
                          output_dir: str,
                          quant_type: str = "fp8",
                          max_batch_size: int = 8,
                          max_input_len: int = 2048,
                          max_output_len: int = 2048,
                          tp_size: int = 1,
                          use_paged_kv: bool = True) -> str:
    """
    Generate a shell script for building a TensorRT-LLM engine.
    """
    
    # Quantization-specific flags
    quant_flags = {
        "fp16": "--dtype float16",
        "bf16": "--dtype bfloat16",
        "fp8": "--dtype bfloat16 --use_fp8_context_fmha enable --quantization fp8",
        "int8": "--dtype float16 --use_smooth_quant --per_token --per_channel",
        "int4_awq": "--dtype float16 --use_weight_only --weight_only_precision int4_awq",
        "nvfp4": "--dtype bfloat16 --quantization nvfp4",
    }
    
    quant_flag = quant_flags.get(quant_type, quant_flags["bf16"])
    
    script = f'''#!/bin/bash
# TensorRT-LLM Engine Build Script
# Model: {model_name}
# Quantization: {quant_type}
# Generated for DGX Spark

set -e

MODEL_PATH="{model_path}"
OUTPUT_DIR="{output_dir}"
CHECKPOINT_DIR="$OUTPUT_DIR/checkpoint"
ENGINE_DIR="$OUTPUT_DIR/engine"

echo "🔨 Building TensorRT-LLM Engine"
echo "   Model: {model_name}"
echo "   Quantization: {quant_type}"
echo "   TP Size: {tp_size}"

# Step 1: Convert checkpoint
echo "\n📦 Step 1: Converting checkpoint..."
python3 convert_checkpoint.py \\
    --model_dir "$MODEL_PATH" \\
    --output_dir "$CHECKPOINT_DIR" \\
    --tp_size {tp_size} \\
    {quant_flag}

# Step 2: Build engine
echo "\n⚙️ Step 2: Building TensorRT engine..."
trtllm-build \\
    --checkpoint_dir "$CHECKPOINT_DIR" \\
    --output_dir "$ENGINE_DIR" \\
    --max_batch_size {max_batch_size} \\
    --max_input_len {max_input_len} \\
    --max_seq_len {max_input_len + max_output_len} \\
    --gemm_plugin auto \\
    --gpt_attention_plugin auto \\
    {"--paged_kv_cache enable" if use_paged_kv else ""} \\
    --remove_input_padding enable \\
    --enable_xqa enable \\
    --use_fused_mlp enable \\
    --workers {tp_size}

# Step 3: Verify build
echo "\n✅ Step 3: Verifying engine..."
python3 -c "
import tensorrt_llm
from tensorrt_llm import ModelRunner

runner = ModelRunner.from_dir('$ENGINE_DIR')
print(f'Engine loaded successfully!')
print(f'Max batch size: {{runner.max_batch_size}}')
print(f'Max input length: {{runner.max_input_len}}')
"

echo "\n🎉 Build complete! Engine saved to: $ENGINE_DIR"
'''
    
    return script


# Generate example scripts
print("📝 TensorRT-LLM Build Script Generator")
print("=" * 60)

# Example 1: Llama 8B with FP8
script_8b = generate_build_script(
    model_name="Llama-3.1-8B-Instruct",
    model_path="/models/Meta-Llama-3.1-8B-Instruct",
    output_dir="/engines/llama-8b-fp8",
    quant_type="fp8",
    max_batch_size=16,
    max_input_len=4096,
    max_output_len=2048
)

print("\n📋 Example Build Script (Llama 8B, FP8):")
print("-" * 60)
print(script_8b[:1500] + "\n...")

# Key optimization flags explained
print("\n🔧 Key Optimization Flags:")
print("""
┌────────────────────────────┬────────────────────────────────────────┐
│ Flag                       │ Purpose                                │
├────────────────────────────┼────────────────────────────────────────┤
│ --gemm_plugin auto         │ Optimized matrix multiplication        │
│ --gpt_attention_plugin     │ Fused attention kernels                │
│ --paged_kv_cache enable    │ PagedAttention for memory efficiency   │
│ --remove_input_padding     │ Skip padding tokens (faster prefill)   │
│ --enable_xqa enable        │ Cross-query attention optimization     │
│ --use_fused_mlp enable     │ Fused MLP layers                       │
│ --use_fp8_context_fmha     │ FP8 flash attention (Blackwell)        │
└────────────────────────────┴────────────────────────────────────────┘
""")

---

## Exercise 3: Benchmark TensorRT-LLM vs Other Engines

**Task**: Create a comprehensive benchmarking framework for comparing inference engines.

In [ ]:
@dataclass
class BenchmarkResult:
    """Results from benchmarking an inference engine."""
    engine: str
    model: str
    batch_size: int
    input_len: int
    output_len: int
    ttft_ms: float
    tpot_ms: float  # Time per output token
    throughput_tps: float
    memory_gb: float
    latency_p99_ms: float


class InferenceEngineBenchmark:
    """
    Benchmark framework for comparing inference engines.
    """
    
    # Simulated baseline metrics (would be measured in real benchmark)
    BASELINE_METRICS = {
        "tensorrt-llm": {
            "ttft_factor": 0.7,    # 30% faster TTFT than baseline
            "tpot_factor": 0.6,    # 40% faster decode
            "memory_factor": 0.8,  # 20% less memory
        },
        "vllm": {
            "ttft_factor": 1.0,    # Baseline
            "tpot_factor": 1.0,
            "memory_factor": 1.0,
        },
        "sglang": {
            "ttft_factor": 0.85,   # 15% faster with RadixAttention
            "tpot_factor": 0.9,
            "memory_factor": 0.95,
        },
        "ollama": {
            "ttft_factor": 1.3,    # Slower (optimized for ease of use)
            "tpot_factor": 1.2,
            "memory_factor": 1.1,
        },
    }
    
    def __init__(self, model_name: str = "llama-8b"):
        self.model = MODELS[model_name]
        # Baseline metrics for 8B model
        self.base_ttft_ms = 50  # 50ms TTFT baseline
        self.base_tpot_ms = 15  # 15ms per token baseline
        self.base_memory_gb = 16  # 16GB baseline
    
    def run_benchmark(self, engine: str, 
                      batch_size: int = 1,
                      input_len: int = 512,
                      output_len: int = 256) -> BenchmarkResult:
        """
        Simulate a benchmark run for an engine.
        """
        metrics = self.BASELINE_METRICS.get(engine, self.BASELINE_METRICS["vllm"])
        
        # Scale by batch size and sequence length
        batch_scale = 1 + (batch_size - 1) * 0.1  # ~10% overhead per batch item
        seq_scale = 1 + (input_len / 1024) * 0.2  # ~20% overhead per 1K tokens
        
        ttft_ms = self.base_ttft_ms * metrics["ttft_factor"] * seq_scale
        tpot_ms = self.base_tpot_ms * metrics["tpot_factor"] * batch_scale
        
        # Calculate throughput
        total_time_ms = ttft_ms + (output_len * tpot_ms)
        total_tokens = batch_size * output_len
        throughput_tps = (total_tokens / total_time_ms) * 1000
        
        # Memory usage
        memory_gb = self.base_memory_gb * metrics["memory_factor"] * (batch_size ** 0.5)
        
        # P99 latency (add some variance)
        latency_p99_ms = total_time_ms * 1.3
        
        return BenchmarkResult(
            engine=engine,
            model=self.model.name,
            batch_size=batch_size,
            input_len=input_len,
            output_len=output_len,
            ttft_ms=ttft_ms,
            tpot_ms=tpot_ms,
            throughput_tps=throughput_tps,
            memory_gb=memory_gb,
            latency_p99_ms=latency_p99_ms
        )
    
    def compare_engines(self, engines: List[str],
                        batch_sizes: List[int] = [1, 8, 32]) -> Dict[str, List[BenchmarkResult]]:
        """
        Compare multiple engines across different batch sizes.
        """
        results = {engine: [] for engine in engines}
        
        for engine in engines:
            for batch_size in batch_sizes:
                result = self.run_benchmark(engine, batch_size=batch_size)
                results[engine].append(result)
        
        return results


# Run comparison benchmark
print("📊 Inference Engine Comparison Benchmark")
print("=" * 80)

benchmark = InferenceEngineBenchmark("llama-8b")
engines = ["tensorrt-llm", "vllm", "sglang", "ollama"]

# Single request benchmark
print("\n📋 Single Request Performance (batch=1, input=512, output=256):")
print("-" * 80)
print(f"{'Engine':<15} {'TTFT':>10} {'TPOT':>10} {'Throughput':>12} {'Memory':>10} {'P99':>12}")
print("-" * 80)

single_results = []
for engine in engines:
    result = benchmark.run_benchmark(engine, batch_size=1)
    single_results.append(result)
    
    print(f"{engine:<15} {result.ttft_ms:>8.1f}ms {result.tpot_ms:>8.1f}ms "
          f"{result.throughput_tps:>10.1f}t/s {result.memory_gb:>8.1f}GB "
          f"{result.latency_p99_ms:>10.1f}ms")

# High throughput benchmark
print("\n📋 High Throughput Performance (batch=32):")
print("-" * 80)
print(f"{'Engine':<15} {'Throughput':>15} {'Memory':>12} {'Improvement':>15}")
print("-" * 80)

baseline_tps = None
for engine in engines:
    result = benchmark.run_benchmark(engine, batch_size=32)
    
    if baseline_tps is None:
        baseline_tps = result.throughput_tps
        improvement = "(baseline)"
    else:
        imp = (result.throughput_tps / baseline_tps - 1) * 100
        improvement = f"{'+' if imp > 0 else ''}{imp:.1f}%"
    
    print(f"{engine:<15} {result.throughput_tps:>13.1f}t/s {result.memory_gb:>10.1f}GB "
          f"{improvement:>15}")

print("\n💡 Key Insights:")
print("   • TensorRT-LLM: Best raw performance (optimized CUDA kernels)")
print("   • vLLM: Great balance of performance and ease of use")
print("   • SGLang: Best for workloads with prefix sharing")
print("   • Ollama: Best for development/prototyping")

---

## Exercise 4: Optimize for Specific Workloads

**Task**: Create workload-specific optimization profiles.

In [ ]:
@dataclass
class WorkloadProfile:
    """Profile for a specific workload type."""
    name: str
    description: str
    avg_input_len: int
    avg_output_len: int
    batch_size: int
    latency_priority: bool  # True = optimize latency, False = optimize throughput
    streaming: bool

WORKLOAD_PROFILES = {
    "chatbot": WorkloadProfile(
        name="Interactive Chatbot",
        description="Low-latency conversational AI",
        avg_input_len=256,
        avg_output_len=128,
        batch_size=1,
        latency_priority=True,
        streaming=True
    ),
    "batch_processing": WorkloadProfile(
        name="Batch Processing",
        description="High-throughput document processing",
        avg_input_len=2048,
        avg_output_len=512,
        batch_size=64,
        latency_priority=False,
        streaming=False
    ),
    "code_completion": WorkloadProfile(
        name="Code Completion",
        description="IDE code suggestions",
        avg_input_len=512,
        avg_output_len=64,
        batch_size=1,
        latency_priority=True,
        streaming=True
    ),
    "summarization": WorkloadProfile(
        name="Document Summarization",
        description="Long document analysis",
        avg_input_len=8192,
        avg_output_len=256,
        batch_size=8,
        latency_priority=False,
        streaming=False
    ),
}


def generate_optimization_config(profile: WorkloadProfile) -> dict:
    """
    Generate TensorRT-LLM optimization config for a workload.
    """
    config = {
        "workload": profile.name,
        "build_config": {},
        "runtime_config": {},
        "recommendations": []
    }
    
    # Build configuration
    if profile.latency_priority:
        # Optimize for low latency
        config["build_config"] = {
            "max_batch_size": min(8, profile.batch_size * 2),
            "max_input_len": profile.avg_input_len * 2,
            "max_output_len": profile.avg_output_len * 2,
            "opt_batch_size": profile.batch_size,
            "builder_opt_level": 5,  # Maximum optimization
            "strongly_typed": True,
        }
        config["recommendations"].append("Use FP8 for best latency on Blackwell")
        config["recommendations"].append("Enable speculative decoding for further speedup")
    else:
        # Optimize for throughput
        config["build_config"] = {
            "max_batch_size": profile.batch_size * 2,
            "max_input_len": profile.avg_input_len,
            "max_output_len": profile.avg_output_len,
            "opt_batch_size": profile.batch_size,
            "builder_opt_level": 4,
            "strongly_typed": True,
        }
        config["recommendations"].append("Use INT4 quantization for maximum throughput")
        config["recommendations"].append("Enable paged KV cache for memory efficiency")
    
    # Runtime configuration
    config["runtime_config"] = {
        "kv_cache_free_gpu_mem_fraction": 0.85 if profile.latency_priority else 0.95,
        "enable_chunked_context": profile.avg_input_len > 2048,
        "streaming": profile.streaming,
        "max_tokens_in_paged_kv_cache": None,  # Auto-calculate
    }
    
    # Additional recommendations based on workload
    if profile.avg_input_len > 4096:
        config["recommendations"].append("Enable chunked prefill for long inputs")
    if profile.streaming:
        config["recommendations"].append("Use streaming tokenizer for faster TTFT")
    
    return config


# Generate optimization configs for each workload
print("🎯 Workload-Specific Optimization Profiles")
print("=" * 70)

for name, profile in WORKLOAD_PROFILES.items():
    config = generate_optimization_config(profile)
    
    print(f"\n📋 {profile.name}")
    print(f"   {profile.description}")
    print(f"   Input/Output: {profile.avg_input_len}/{profile.avg_output_len} tokens")
    print(f"   Batch size: {profile.batch_size}")
    print(f"   Priority: {'Latency' if profile.latency_priority else 'Throughput'}")
    
    print("\n   Build Config:")
    for key, value in config["build_config"].items():
        print(f"      {key}: {value}")
    
    print("\n   Recommendations:")
    for rec in config["recommendations"]:
        print(f"      • {rec}")
    
    print("-" * 70)

---

## Key Takeaways

1. **Quantization Trade-offs**:
   - FP16/BF16: Best quality, highest memory
   - FP8: Great balance (Blackwell native)
   - INT4: Maximum throughput, slight quality loss

2. **Build Optimization**:
   - Use all available plugins (GEMM, attention, MLP fusion)
   - Enable paged KV cache for memory efficiency
   - Set appropriate batch/sequence limits for your workload

3. **TensorRT-LLM Advantages**:
   - ~30-40% faster than other engines
   - Native Blackwell optimizations
   - Best for production deployments

4. **Workload Optimization**:
   - Chatbot: Prioritize TTFT, use FP8
   - Batch: Prioritize throughput, use INT4
   - Long context: Enable chunked prefill